<a href="https://colab.research.google.com/github/Ashi743/deep-learning-/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df= pd.read_csv("/content/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()["Outcome"]

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
ss= StandardScaler()
X =ss.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [58]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [16]:
model= Sequential()
model.add(Dense(32, activation= "relu", input_dim= 8 ) )
model.add(Dense(1, activation= "sigmoid"))

model.compile(optimizer= "adam", loss= "binary_crossentropy", metrics= ["accuracy"])
model.fit(X_train,y_train, batch_size= 32, epochs= 10, validation_data= (X_test,y_test) )

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4321 - loss: 0.8381 - val_accuracy: 0.5260 - val_loss: 0.7859
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5525 - loss: 0.7329 - val_accuracy: 0.6104 - val_loss: 0.7151
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6030 - loss: 0.6695 - val_accuracy: 0.6169 - val_loss: 0.6656
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6628 - loss: 0.6197 - val_accuracy: 0.6623 - val_loss: 0.6273
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6942 - loss: 0.5878 - val_accuracy: 0.7078 - val_loss: 0.5972
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7246 - loss: 0.5450 - val_accuracy: 0.7403 - val_loss: 0.5770
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7540 - loss: 0.5228 - val_accuracy: 0.7468 - val_loss: 0.5614
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7563 - loss: 0.5076 - val_accuracy: 0.7532 - val_loss

to tune  

In [54]:
# how to select appropriate  optimizer
# No. of nodes in a layer
# No. of layers
# all in all one model

In [9]:
 pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [10]:
import keras_tuner as kt

In [17]:
def build_model(hp):
  model= Sequential()
  model.add(Dense(32, activation= "relu", input_dim= 8 ) )
  model.add(Dense(1, activation= "sigmoid") )
  optimizer= hp.Choice("optimizer", values= [ "sgd", "rmsprop", "adadelta"])
  model.compile(optimizer= optimizer, loss= "binary_crossentropy", metrics= ["accuracy"])
  return model

In [19]:
tuner =kt.RandomSearch(build_model, objective='val_accuracy',
                       max_trials=3,
                       directory= 'mkdir',
                       project_name= 'srk')

In [20]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test) )

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 08s


In [21]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [22]:
model= tuner.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(X_train,y_train, batch_size= 32, initial_epoch=6, epochs= 100, validation_data= (X_test,y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7490 - loss: 0.5262 - val_accuracy: 0.6883 - val_loss: 0.5310
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7384 - loss: 0.5124 - val_accuracy: 0.7208 - val_loss: 0.5209
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7922 - loss: 0.4793 - val_accuracy: 0.7468 - val_loss: 0.5143
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7762 - loss: 0.4778 - val_accuracy: 0.7468 - val_loss: 0.5096
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7423 - loss: 0.4901 - val_accuracy: 0.7532 - val_loss: 0.5061
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7871 - loss: 0.4624 - val_accuracy: 0.7468 - val_loss: 0.5032
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7768 - loss: 0.4710 - val_accuracy: 0.7403 - val_loss: 0.5020
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7784 - loss: 0.4678 - val_accuracy: 0.74

In [11]:
def build_model1(hp):
  model= Sequential()

  units= hp.Int("units", min_value=8, max_value=128, step=8)

  model.add(Dense(units= units, activation= "relu", input_dim= 8 ) )
  model.add(Dense(1, activation= "sigmoid"))

  #optimizer= hp.Choice("optimizer", values= ["adam", "sgd", "rmsprop", "adadelta"])

  model.compile(optimizer= 'adam', loss= "binary_crossentropy", metrics= ["accuracy"])
  return model

In [12]:
tuner1= kt.RandomSearch(build_model1, objective= "val_accuracy", max_trials=5)
tuner1.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test) )


Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 21s


{'units': 32}

In [13]:
tuner1.get_best_hyperparameters()[0].values

{'units': 32}

In [14]:
model= tuner1.get_best_models(num_models=1)[0]

In [15]:
model.fit(X_train,y_train, batch_size= 32, initial_epoch=6, epochs= 100, validation_data= (X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.7761 - loss: 0.5294 - val_accuracy: 0.7922 - val_loss: 0.5193
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7431 - loss: 0.5243 - val_accuracy: 0.7792 - val_loss: 0.5132
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7705 - loss: 0.4883 - val_accuracy: 0.7792 - val_loss: 0.5068
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7698 - loss: 0.4873 - val_accuracy: 0.7727 - val_loss: 0.5049
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7771 - loss: 0.4782 - val_accuracy: 0.7662 - val_loss: 0.5034
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7541 - loss: 0.5035 - val_accuracy: 0.7662 - val_loss: 0.5008
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7407 - loss: 0.4822 - val_accuracy: 0.7597 - val_loss: 0.5007
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7788 - loss: 0.4553 - val_accuracy: 0

In [ ]:
#layers

In [27]:
def build_model_layers(hp):
  model= Sequential()

  model.add(Dense(72, activation= "relu", input_dim= 8 ) )
  #layers
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(units= 72, activation= "relu", input_dim= 8))

  model.add(Dense(1, activation= "sigmoid") )

  optimizer= hp.Choice("optimizer", values= [ "sgd", "rmsprop", "adadelta"])

  model.compile(optimizer= optimizer, loss= "binary_crossentropy", metrics= ["accuracy"])
  return model

tunerqwe= kt.RandomSearch(build_model_layers, objective= "val_accuracy", max_trials=5)
tunerqwe.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test) )


Reloading Tuner from ./untitled_project/tuner0.json


In [36]:
tunerqwe.get_best_hyperparameters()[0].values

{'units': 32, 'num_layers': 1, 'optimizer': 'sgd'}

In [40]:
def build_model_layers(hp):
  model= Sequential()

  model.add(Dense(72, activation= "relu", input_shape= (8,) ) ) # Correcting input shape to (8,)
  #layers
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(units= 72, activation= "relu")) # Removing unnecessary input_dim

  model.add(Dense(1, activation= "sigmoid") )

  optimizer= hp.Choice("optimizer", values= [ "sgd", "rmsprop", "adadelta"])

  model.compile(optimizer= optimizer, loss= "binary_crossentropy", metrics= ["accuracy"])
  return model

tunerqwe= kt.RandomSearch(build_model_layers, objective= "val_accuracy", max_trials=5)
tunerqwe.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test) )

tunerqwe.get_best_hyperparameters()[0].values



Reloading Tuner from ./untitled_project/tuner0.json


{'units': 32}

In [41]:
model = tunerqwe.get_best_models(num_models=1)[0]
model.fit(X_train,y_train, batch_size= 32, initial_epoch=6, epochs= 100, validation_data= (X_test,y_test))

ValueError: A total of 3 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 72), Received: value.shape=(8, 32). Target variable: <KerasVariable shape=(8, 72), dtype=float32, path=sequential/dense/kernel>

List of objects that could not be loaded:
[<Dense name=dense, built=True>, <Dense name=dense_1, built=True>, <Dense name=dense_2, built=True>]

In [67]:
def build_model(hp):
    model = Sequential()

    # Add hidden layers based on hyperparameters
    for i in range(hp.Int('num_layers', min_value=2, max_value=20, step=2)):
        if i == 0:
            # First layer with input dimension
            model.add(
                Dense(
                    units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                    input_dim=8
                )
            )
            # Add Dropout
            model.add(Dropout(hp.Float('dropout_rate' + str(i), min_value=0.0, max_value=0.9, step=0.1)))
        else:
            # Subsequent layers without input dimension
            model.add(
                Dense(
                    units=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
                )
            )
            # Add Dropout
            model.add(Dropout(hp.Float('dropout_rate' + str(i), min_value=0.0, max_value=0.9, step=0.1)))

    # Output layer
    model.add(Dense(1, activation="sigmoid"))

    # Compile the model with hyperparameter tuning for optimizer
    model.compile(optimizer=hp.Choice("optimizer", values=['adam', 'sgd', 'nadam', 'adadelta', 'rmsprop']),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    return model


In [68]:
tuner = kt.RandomSearch(
    build_model,                      # The model-building function
    objective="val_accuracy",          # The metric to optimize
    max_trials=5,                      # The maximum number of trials
    directory='my_dir',                # Correct spelling of directory
    project_name='my_project2'          # The name of the project (optional)
)


In [69]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7207792401313782
Total elapsed time: 00h 00m 35s


In [70]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 64,
 'activation0': 'sigmoid',
 'dropout_rate0': 0.1,
 'units1': 32,
 'activation1': 'tanh',
 'dropout_rate1': 0.1,
 'optimizer': 'nadam',
 'units2': 120,
 'activation2': 'relu',
 'dropout_rate2': 0.1,
 'units3': 88,
 'activation3': 'tanh',
 'dropout_rate3': 0.1,
 'units4': 56,
 'activation4': 'tanh',
 'dropout_rate4': 0.30000000000000004,
 'units5': 80,
 'activation5': 'sigmoid',
 'dropout_rate5': 0.7000000000000001,
 'units6': 48,
 'activation6': 'relu',
 'dropout_rate6': 0.4,
 'units7': 80,
 'activation7': 'sigmoid',
 'dropout_rate7': 0.1,
 'units8': 40,
 'activation8': 'tanh',
 'dropout_rate8': 0.6000000000000001,
 'units9': 88,
 'activation9': 'sigmoid',
 'dropout_rate9': 0.5}

In [71]:
model= tuner.get_best_models(num_models=1)[0]

In [72]:
model.fit(X_train,y_train, batch_size= 32, initial_epoch=6, epochs= 100, validation_data= (X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7551 - loss: 0.5441 - val_accuracy: 0.7273 - val_loss: 0.5296
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7479 - loss: 0.5363 - val_accuracy: 0.7273 - val_loss: 0.5143
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7605 - loss: 0.5134 - val_accuracy: 0.7273 - val_loss: 0.5080
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7131 - loss: 0.5656 - val_accuracy: 0.7078 - val_loss: 0.5107
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6957 - loss: 0.5641 - val_accuracy: 0.7468 - val_loss: 0.5150
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7352 - loss: 0.5101 - val_accuracy: 0.7403 - val_loss: 0.5089
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7262 - loss: 0.5688 - val_accuracy: 0.7403 - val_loss: 0.5079
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7335 - loss: 0.5226 - val_accuracy: 0.71